##### Lets research strategies and aggregate their results

In [43]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import ta
import plotly_express as px
import plotly.graph_objects as go
import yfinance as yf
from yahoo_fin import stock_info as si 
from pandas_datareader import DataReader
import requests
%matplotlib inline

- Please double check me on these and lmk if they're wrong

### RSI Strategy
If RSI crosses above 30: buy. If RSI crosses below 70: sell

### CCI Strategy
If CCI crosses above -100: buy. If CCI crosses below 100: sell

### Yahoo Finance Rating
1-2: Buy
2-3: Hold
3-5: Sell

In [53]:
# A class containing all of the stock indicators we can think of and their aggregate ratings (the score)
class StockScore:
    # Constructor to download stock data from yf
    def __init__(self,stock):
        self.stock = stock
        self.data = yf.download(stock)
        self.ratings = []
        # Calculate technical indicators
        self.data = ta.add_all_ta_features(self.data,open='Open',high='High',low='Low',
                                           close='Close',volume='Volume',fillna=True)
    # Rsi buy/sell signal
    def rsi_signal(self):
        rsi = self.data['momentum_rsi']
        if rsi[-1] > 30 and rsi[-3] < 30:
            self.ratings.append(1)
            print("RSI:Buy")
        elif rsi[-1] < 70  and rsi[-3] > 70:
            self.ratings.append(0)
            print("RSI:Sell")
        else:
            self.ratings.append(.5)
            print("RSI:Hold")
            
    # CCI buy/sell signal
    def cci_signal(self):
        cci = self.data['trend_cci']
        if (cci[-1] > -100 and cci[-1] < -50) and cci[-5] < -100:
            self.ratings.append(1)
            print("CCI:Buy")
        elif cci[-1] < 100 and cci[-5] > 100:
            self.ratings.append(0)
            print("CCI:Sell")
        else:
            self.ratings.append(.5)
            print("CCI:Hold")
    # Yahoo finance analyst rating
    def yf_rating(self):
        lhs_url = 'https://query2.finance.yahoo.com/v10/finance/quoteSummary/'
        rhs_url = '?formatted=true&crumb=swg7qs5y9UP&lang=en-US&region=US&' \
                  'modules=upgradeDowngradeHistory,recommendationTrend,' \
                  'financialData,earningsHistory,earningsTrend,industryTrend&' \
                  'corsDomain=finance.yahoo.com'

        url =  lhs_url + self.stock + rhs_url
        r = requests.get(url)
        if not r.ok:
            recommendation = 6
        try:
            result = r.json()['quoteSummary']['result'][0]
            recommendation =result['financialData']['recommendationMean']['fmt']
        except:
            recommendation = 6

        recommendation = float(recommendation)
        print (f"{self.stock} Yahoo Finance Analyst Rating:{recommendation}")
        
        # Scoring
        if recommendation >= 1 and recommendation < 2:
            self.ratings.append(1)
            print("YF Rating:Buy")
        elif recommendation >=2 and recommendation < 3:
            self.ratings.append(.5)
            print("YF Rating:Hold")
        else:
            self.ratings.append(0)
            print("YF Rating:Sell")
    
        
    # Stock score
    def score(self):
        # Run functions
        self.cci_signal()
        self.rsi_signal()
        self.yf_rating()
        return np.mean(self.ratings)

In [72]:
stock1 = StockScore('CRK')
stock2 = StockScore('SIRI')

[*********************100%***********************]  1 of 1 completed


/Users/HarrisonHoffman/opt/anaconda3/lib/python3.7/site-packages/ta/trend.py:543: RuntimeWarning:

invalid value encountered in double_scalars

/Users/HarrisonHoffman/opt/anaconda3/lib/python3.7/site-packages/ta/trend.py:547: RuntimeWarning:

invalid value encountered in double_scalars



[*********************100%***********************]  1 of 1 completed


In [74]:
stock1.score()

CCI:Buy
RSI:Hold
CRK Yahoo Finance Analyst Rating:2.2
YF Rating:Hold


0.6666666666666666

In [75]:
stock2.score()

CCI:Hold
RSI:Hold
SIRI Yahoo Finance Analyst Rating:2.3
YF Rating:Hold


0.5